## Phishing Site Detection

In the Pre processing of the Data the attributes are coded into following format for application of different Classifiers 

 	SFH's type is nominal, range is ('1', '-1', '0')
 	popUpWidnow's type is nominal, range is ('-1', '0', '1')
 	SSLfinal_State's type is nominal, range is ('1', '-1', '0')
 	Request_URL's type is nominal, range is ('-1', '0', '1')
 	URL_of_Anchor's type is nominal, range is ('-1', '0', '1')
 	web_traffic's type is nominal, range is ('1', '0', '-1')
 	URL_Length's type is nominal, range is ('1', '-1', '0')
 	age_of_domain's type is nominal, range is ('1', '-1')
 	having_IP_Address's type is nominal, range is ('0', '1')
 	Result's type is nominal, range is ('0', '1', '-1'))
    
    The  data coding is as follows:
    "1"  - Legitimate Website
    "0"  - Suspicious
    "-1" - Malicious Website

In [1]:
your_local_path = 'C:/Users/meghna/Desktop/My_Project/Phishing Site Detection/'

In [3]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from scipy.io.arff import loadarff
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

In [4]:
phishing_data_raw = loadarff(your_local_path+"PhishingData.arff")

In [5]:
phishing_data_raw

(array([(b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', b'1', b'0', b'0'),
        (b'-1', b'-1', b'-1', b'-1', b'-1', b'0', b'1', b'1', b'1', b'1'),
        (b'1', b'-1', b'0', b'0', b'-1', b'0', b'-1', b'1', b'0', b'1'),
        ...,
        (b'-1', b'0', b'-1', b'-1', b'-1', b'0', b'-1', b'-1', b'0', b'1'),
        (b'0', b'0', b'1', b'0', b'0', b'0', b'-1', b'1', b'0', b'1'),
        (b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'-1', b'0', b'-1')],
       dtype=[('SFH', 'S2'), ('popUpWidnow', 'S2'), ('SSLfinal_State', 'S2'), ('Request_URL', 'S2'), ('URL_of_Anchor', 'S2'), ('web_traffic', 'S2'), ('URL_Length', 'S2'), ('age_of_domain', 'S2'), ('having_IP_Address', 'S1'), ('Result', 'S2')]),
 Dataset: Phishing
 	SFH's type is nominal, range is ('1', '-1', '0')
 	popUpWidnow's type is nominal, range is ('-1', '0', '1')
 	SSLfinal_State's type is nominal, range is ('1', '-1', '0')
 	Request_URL's type is nominal, range is ('-1', '0', '1')
 	URL_of_Anchor's type is nominal, range is ('-1',

In [6]:
# Selection of the Data and converting data into numpy format for flexibility in cleaning
phishing_data_array = np.array(phishing_data_raw[0])
phishing_data_array

array([(b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', b'1', b'0', b'0'),
       (b'-1', b'-1', b'-1', b'-1', b'-1', b'0', b'1', b'1', b'1', b'1'),
       (b'1', b'-1', b'0', b'0', b'-1', b'0', b'-1', b'1', b'0', b'1'),
       ...,
       (b'-1', b'0', b'-1', b'-1', b'-1', b'0', b'-1', b'-1', b'0', b'1'),
       (b'0', b'0', b'1', b'0', b'0', b'0', b'-1', b'1', b'0', b'1'),
       (b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'-1', b'0', b'-1')],
      dtype=[('SFH', 'S2'), ('popUpWidnow', 'S2'), ('SSLfinal_State', 'S2'), ('Request_URL', 'S2'), ('URL_of_Anchor', 'S2'), ('web_traffic', 'S2'), ('URL_Length', 'S2'), ('age_of_domain', 'S2'), ('having_IP_Address', 'S1'), ('Result', 'S2')])

In [7]:
# Converting the numpy array into Pandas data frame  and casting the coloumns to numeric type
phishing_df = pd.DataFrame(phishing_data_array).apply(pd.to_numeric)
print(phishing_df.shape)
phishing_df

(1353, 10)


,SFH,popUpWidnow,SSLfinal_State,Request_URL,URL_of_Anchor,web_traffic,URL_Length,age_of_domain,having_IP_Address,Result
0,1,-1,1,-1,-1,1,1,1,0,0
1,-1,-1,-1,-1,-1,0,1,1,1,1
2,1,-1,0,0,-1,0,-1,1,0,1
3,1,0,1,-1,-1,0,1,1,0,0
4,-1,-1,1,-1,0,0,-1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1348,-1,-1,-1,-1,-1,-1,0,1,0,1
1349,-1,0,1,0,-1,0,0,1,0,-1
1350,-1,0,-1,-1,-1,0,-1,-1,0,1
1351,0,0,1,0,0,0,-1,1,0,1


In [8]:
# Getting column names and joining them.
col_names = phishing_df.columns
print(col_names)
print("Column names: " + ", ".join(phishing_df.columns))

# The scikit learn package takes data in form of labels and predictors
# Extracting Result column to pass as a target column hence we are storing its values in labels
labels = phishing_df["Result"].values
print(labels)

Index(['SFH', 'popUpWidnow', 'SSLfinal_State', 'Request_URL', 'URL_of_Anchor',
       'web_traffic', 'URL_Length', 'age_of_domain', 'having_IP_Address',
       'Result'],
      dtype='object')
Column names: SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain, having_IP_Address, Result
[ 0  1  1 ...  1  1 -1]


In [9]:
predictor_cols = col_names[0:len(col_names)-2] # Selecting the predictor column names
print("Predictor_Columns:" + ", ".join(predictor_cols))
#Extracting Predictor points 
predictors = phishing_df[predictor_cols].values
print(predictors)

Predictor_Columns:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain
[[ 1 -1  1 ...  1  1  1]
 [-1 -1 -1 ...  0  1  1]
 [ 1 -1  0 ...  0 -1  1]
 ...
 [-1  0 -1 ...  0 -1 -1]
 [ 0  0  1 ...  0 -1  1]
 [ 1  0  1 ...  0 -1 -1]]


###### Next, we'll make a model that will be used to predict patients. In this case, we will use different algorithms.

### k-Nearest Neighbours

In [10]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(predictors, labels, test_size=0.3, random_state=4)

In [11]:
#applying grid search to find best performing parameters 
k = np.arange(1,31,1) 
parameters = {'n_neighbors' : k}
clf_knn = KNeighborsClassifier(weights = 'distance')
clf = GridSearchCV(clf_knn,parameters,cv=13)
clf_knn = clf.fit(X_train,y_train)
print(clf_knn.best_params_)

{'n_neighbors': 8}


In [12]:
#Prediction
y_pred = clf_knn.predict(X_test)
target_names = ['-1','0','1']

#Classification Report
print(classification_report(y_test, y_pred, target_names=target_names))

#Confusion Matrix
print(confusion_matrix(y_test, y_pred))

#Accuracy
print("Accuracy: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.93      0.88      0.90       216
           0       0.71      0.81      0.76        27
           1       0.86      0.90      0.88       163

    accuracy                           0.88       406
   macro avg       0.83      0.86      0.85       406
weighted avg       0.89      0.88      0.88       406

[[190   4  22]
 [  3  22   2]
 [ 12   5 146]]
Accuracy:  0.8817733990147784


###### Now from the classification report, we can see precision has been increased for the metrics '0' & '1' and slightly decreased for '-1' but as overall performance of model is good hence we can say our model will predict just fine.

###### With our model, we achieved 88.67% accuracy. By finetuning the hyper parameters of KNN Classifier, we can achieve more accuracy.

In [13]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Converting to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Creating new KNN object
knn = KNeighborsClassifier()

#Use GridSearch
classifier = GridSearchCV(knn, hyperparameters, cv=10)

#Fitting the model
best_model = classifier.fit(X_train,y_train)

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 14
Best p: 1
Best n_neighbors: 10


In [14]:
#KNN Prediction
y_pred = classifier.predict(X_test)
target_names = ['-1','0','1']

#KNN Classification Report
print(classification_report(y_test, y_pred, target_names=target_names))

#KNN Confusion Matrix
print(confusion_matrix(y_test, y_pred))

#KNN Accuracy
knn_accuracy = accuracy_score(y_test, y_pred)
print("KNN Accuracy: ", knn_accuracy)

              precision    recall  f1-score   support

          -1       0.90      0.91      0.91       216
           0       0.71      0.44      0.55        27
           1       0.88      0.93      0.90       163

    accuracy                           0.88       406
   macro avg       0.83      0.76      0.78       406
weighted avg       0.88      0.88      0.88       406

[[196   2  18]
 [ 12  12   3]
 [  9   3 151]]
KNN Accuracy:  0.8842364532019704


### Random Forest Classifier

In [15]:
#Random Forest
parameters = [{'n_estimators': [100, 700],
    'max_features': ['sqrt', 'log2'],
    'criterion' :['gini', 'entropy']}]

grid_search = GridSearchCV(RandomForestClassifier(),  parameters,cv =5, n_jobs= -1)
grid_search.fit(X_train, y_train)

#printing best parameters 
print("Best Accurancy =" +str( grid_search.best_score_))
print("best parameters =" + str(grid_search.best_params_)) 

#fitting RandomForest regression with best params 
classifier = RandomForestClassifier(n_estimators = 100, criterion = "gini", max_features = 'log2',  random_state = 0)
classifier.fit(X_train, y_train)

Best Accurancy =0.8923029796714006
best parameters ={'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 700}


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
#predicting the tests set result
y_pred = classifier.predict(X_test)

#Random Forest Classification Report
print(classification_report(y_test, y_pred, target_names=target_names))

#Random Forest Confusion Matrix
print(confusion_matrix(y_test, y_pred))

#Random Forest Accuracy
random_forest_accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Accuracy: ", random_forest_accuracy)

              precision    recall  f1-score   support

          -1       0.95      0.89      0.92       216
           0       0.77      0.85      0.81        27
           1       0.88      0.93      0.90       163

    accuracy                           0.90       406
   macro avg       0.86      0.89      0.88       406
weighted avg       0.91      0.90      0.90       406

[[192   4  20]
 [  3  23   1]
 [  8   3 152]]
Random Forest Accuracy:  0.9039408866995073


### Support-vector Machines (SVM)

In [17]:
#applying grid search to find best performing parameters 
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 10, 100, 1000], 'gamma': [ 0.1, 0.2,0.3, 0.5]}]
grid_search = GridSearchCV(SVC(kernel='rbf' ),  parameters,cv =5, n_jobs= -1)
grid_search.fit(X_train, y_train)

#printing best parameters 
print("Best Accurancy =" +str( grid_search.best_score_))
print("best parameters =" + str(grid_search.best_params_)) 

#fitting kernel SVM  with best parameters calculated 

classifier = SVC(C=1000, kernel = 'rbf', gamma = 0.2 , random_state = 0)
classifier.fit(X_train, y_train)

Best Accurancy =0.8880757449178501
best parameters ={'C': 10, 'gamma': 0.1}


SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [18]:
#predicting the tests set result
y_pred = classifier.predict(X_test)

#SVM Classification Report
print(classification_report(y_test, y_pred, target_names=target_names))

#SVM Confusion Matrix
print(confusion_matrix(y_test, y_pred))

#SVM Accuracy
svm_accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy: ",svm_accuracy)

              precision    recall  f1-score   support

          -1       0.94      0.88      0.91       216
           0       0.67      0.81      0.73        27
           1       0.87      0.91      0.89       163

    accuracy                           0.89       406
   macro avg       0.83      0.87      0.84       406
weighted avg       0.89      0.89      0.89       406

[[190   6  20]
 [  3  22   2]
 [  9   5 149]]
SVM Accuracy:  0.8891625615763546


### Logistic Regression

In [19]:
#fitting logistic regression 
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
#predicting the tests set result
y_pred = classifier.predict(X_test)

#Logistic Regression Classification Report
print(classification_report(y_test, y_pred, target_names=target_names))

#Logistic Regression Confusion Matrix
print(confusion_matrix(y_test, y_pred))

#Logistic Regression Accuracy
logistic_accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy: ", logistic_accuracy)

              precision    recall  f1-score   support

          -1       0.86      0.88      0.87       216
           0       0.50      0.15      0.23        27
           1       0.81      0.88      0.84       163

    accuracy                           0.83       406
   macro avg       0.72      0.64      0.65       406
weighted avg       0.82      0.83      0.82       406

[[190   2  24]
 [ 13   4  10]
 [ 17   2 144]]
Logistic Regression Accuracy:  0.8325123152709359


In [27]:
results = pd.DataFrame({'Model': ['KNN', 'Random Forest', 'Support Vector Machines', 'Logistic Regression'],
    'Score': [knn_accuracy*100, random_forest_accuracy*100, 
              svm_accuracy*100, logistic_accuracy*100]})
result_data = results.sort_values(by='Score', ascending=False)
result_data = result_data.set_index('Score')
result_data.head(5)

,Model
Score,
90.394089,Random Forest
88.916256,Support Vector Machines
88.423645,KNN
83.251232,Logistic Regression


#### From above data, we can see that Random Forest has come out to be the best scorer amongst all four but as we know Random Forest usually shows high score for the data it is trained on, hence we can also consider SVM or KNN models and tune them further to their best score. Next up, we can deploy our model using different deployment platforms or approches available to us Google Cloud Platform, Kubernetes, etc.